# Project/Notebook Goals

## Assignment:  
* Find active oil/gas wells within a specified radius for a separate list of inactive wells
* The goal is to determine if the active wells have potentially saved the oil & gas lease pertaining to the inactive well
* Provide client with Excel Pivot Table showcasing viable active wells that could've saved lease

## Code Overview
1.  Read in geospatial data
    * Reference Wells - Shapefile of Inactive wells for which company wants to find active wells wtihin 0.5 mile radius
    * Active Wells - Shapefile of all active wells in counties corresponding to inactive wells
2. Prep data by converting CRS for each data set and adding buffer radius to inactive wells
3. Geospatially filter active wells within buffer radius for each inactive well
    * During this step, each returned geodataframe will be stored in a list for concatenation
    * Also, select fields of the inactive well will be stored to identify list of active wells back to inactive well
4. Determine if active well could've saved lease for inactive well
    * Find minimum first production date for each active well
        * Wells can have multiple zones, and could contain multiple dates for first production date.  Thus, grouping by each well identifier (API/UWI) is required to ensure we get the earliest date
    * Based on last production date of inactive well, and earliest first production date of active well within radius, determine if they were in within 1 year of each other
5. Export source table for creating Pivot Table in Excel for visualizing results


# Import Libraries

In [27]:
import geopandas as gp
import pandas as pd
import numpy as np
import datetime as dt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in Data and Preview

In [28]:
activewells = gp.read_file("zip://Source Data/Active Production/activewells.ZIP")
refwells = gp.read_file("zip://Source Data/Wells to Buffer/bufferwells.ZIP")

activewells.head()
refwells.head()

,APIUWI,OpAlias,LeaseName,WellNo,County,Reservoir,ProdType,ProdStatus,DrillType,TD,...,BELqBFac,BEEURGasMo,BEGasEURpr,BEGasBFac,FulLqEURMo,FulLqBFac,FulGsEURMo,FulGasBFac,OtherNo,geometry
0,35119237270000,CHER OIL INC LTD,MARK BOYLES,1,PAYNE (OK),BARTLESVILLE,OIL,ACTIVE,V,4203.0,...,2.00,406.0,No,2.00,406.0,2.00,406.0,2.00,11911618000000,POINT (-96.89246 36.04677)
1,42479389500000,"FLETCHER OPERATING, LLC",WRIGHT-LONG GAS UNIT,2,WEBB (TX),OLMOS,GAS,ACTIVE,V,7704.0,...,1.70,399.0,No,2.00,399.0,0.68,399.0,0.66,5541,POINT (-99.43596 28.09880)
2,17017336990000,AETHON ENERGY OPERATING LLC,CV RA SU84;CBP 7,001-ALT,CADDO (LA),CV/HOSS,GAS,ACTIVE,D,10800.0,...,2.00,404.0,No,1.80,404.0,0.97,404.0,0.93,231119,POINT (-93.63690 32.38579)
3,42479387450000,"HILCORP ENERGY COMPANY, INC.",PENA-CUEVA GU,1,WEBB (TX),LOBO CONS.,GAS,ACTIVE,D,9626.0,...,1.03,395.0,No,0.96,395.0,0.74,395.0,0.34,0,POINT (-99.08561 27.45505)
4,17017336670000,CHANSE ENERGY CORPORATION,FULLER A,092,CADDO (LA),NACATOSH,OIL,ACTIVE,V,1200.0,...,1.99,0.0,No,0.00,405.0,1.99,0.0,0.00,230912,POINT (-93.89046 32.87569)


,APIUWI,OpAlias,LeaseName,WellNo,County,Reservoir,ProdType,ProdStatus,DrillType,TD,...,BELqBFac,BEEURGasMo,BEGasEURpr,BEGasBFac,FulLqEURMo,FulLqBFac,FulGsEURMo,FulGasBFac,OtherNo,geometry
0,35119227310000,STACY OPERATING LLC,LOZIER,13,PAYNE (OK),MISENER,OIL,INACTIVE,V,3970.0,...,None,None,No,None,None,None,None,None,11905524410000,POINT (-96.84287 36.07648)
1,35119226530000,STACY OPERATING LLC,LOZIER (TWIN),12,PAYNE (OK),WILCOX,OIL,INACTIVE,V,3912.0,...,None,None,No,None,None,None,None,None,11905524410000,POINT (-96.84289 36.07516)
2,35119225680000,STACY OPERATING LLC,LOZIER (TWIN),11,PAYNE (OK),MISENER/WILCOX,OIL,INACTIVE,V,3906.0,...,None,None,No,None,None,None,None,None,11905524410000,POINT (-96.84291 36.07349)
3,35119224510000,STACY OPERATING LLC,DANA-LOZIER (LOZIER #10),10,PAYNE (OK),MISENER/WILCOX,OIL,SPUDDED,V,3914.0,...,None,None,No,None,None,None,None,None,11905524410000,POINT (-96.84340 36.07517)
4,17113208680000,"OLEUM OPERATING COMPANY, L.C.",SAGRERA HEIRS,001,VERMILION (LA),None,GAS,INACTIVE,D,14267.0,...,None,None,None,None,None,None,None,None,302726,POINT (-92.18802 29.84410)


# Prep Data

## Convert Geospatial and Geo Dataframes to Same CRS

In [29]:
activewells.crs
refwells.crs

activewells.to_crs(epsg='26913', inplace=True)
refwells.to_crs(epsg='26913', inplace=True)

activewells.crs
refwells.crs

{'init': 'epsg:4326'}

{'init': 'epsg:4326'}

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

## Add Buffers to Wells for Geospatial Filtering

In [30]:
miradius = 0.5
bufferradius = miradius * 1603.34 # converting to miles to meters

refwells['buffer'] = refwells['geometry'].apply(lambda x: x.buffer(bufferradius))

### Test Filter

In [31]:
activewells.loc[activewells.within(refwells.iloc[0]['buffer'])]

,APIUWI,OpAlias,LeaseName,WellNo,County,Reservoir,ProdType,ProdStatus,DrillType,TD,...,BELqBFac,BEEURGasMo,BEGasEURpr,BEGasBFac,FulLqEURMo,FulLqBFac,FulGsEURMo,FulGasBFac,OtherNo,geometry
4050,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.78,88.0,No,1.85,88.0,1.99,88.0,1.51,11905524400000,POINT (1235260.589 4023024.078)
10040,35119225880000,STACY OPERATING LLC,STATE PENNEY,4,PAYNE (OK),"RED FORK,PRUE,SKINNER,MISS",OIL,ACTIVE,V,4164.0,...,0.26,161.0,Yes,1.69,161.0,0.26,161.0,1.69,11902114900371,POINT (1235871.294 4023615.964)
11722,35119208510000,STACY OPERATING LLC,LOZIER,1,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3940.0,...,2.00,88.0,No,0.97,88.0,2.00,88.0,1.51,11905524400000,POINT (1235184.467 4023424.147)
14974,35119225810000,STACY OPERATING LLC,BUTCHER-JAMES,4,PAYNE (OK),PRUE,OIL,ACTIVE,V,3910.0,...,1.94,166.0,Yes,1.69,166.0,1.94,166.0,1.69,11902780700000,POINT (1235453.446 4022844.164)


# Perform Geospatial Filtering

In [32]:
listofresults = []

for row in range(0, len(refwells)):
    
    print(f"Searching Active Wells for: {refwells['LeaseName'].iloc[row]}")
    
    buffer = refwells['buffer'].iloc[row]
    hbpwells = activewells.loc[activewells.within(buffer)]
              
          
    #creating dataframe with empty row to store track wells with no active wells found in buffer radius       
    if hbpwells.empty: #if no wells found, dataframe will be empty
          
          hbpwells = pd.DataFrame(columns = activewells.columns) #create dataframe with same columns
          hbpwells = hbpwells.append(pd.Series(), ignore_index=True) #adding empty row so we can assign values
          hbpwells["Reference Inactive API"] = refwells['APIUWI'].iloc[row]  # adding reference well api 
          hbpwells["Reference Inactive Well Name"] = refwells["LeaseName"].iloc[row] + " " + refwells['WellNo'].apply(lambda x: str(x)).iloc[row]
          hbpwells['Reference Inactive Well Last Prod Date'] = refwells['LstPrdDate'].iloc[row]
          hbpwells["County"] = refwells["County"].iloc[row]
          hbpwells["LeaseName"] = f'no active wells found in {miradius} mi radius' # assigining values to all columns except last 2 to mention no wells found
    
    #assiging reference well data to rows to understand which filtered wells are within 0.5 miles of reference well      
    hbpwells["Reference Inactive API"] = refwells['APIUWI'].iloc[row]
    hbpwells["Reference Inactive Well Name"] = refwells["LeaseName"].iloc[row] + " " + refwells['WellNo'].apply(lambda x: str(x)).iloc[row]
    hbpwells['Reference Inactive Well Last Prod Date'] = refwells['LstPrdDate'].iloc[row]
    
    #after geodataframe is created with geospatial filter, append to list for concatenation      
    listofresults.append(hbpwells)

len(listofresults)

Searching Active Wells for: LOZIER


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Searching Active Wells for: LOZIER (TWIN)
Searching Active Wells for: LOZIER (TWIN)
Searching Active Wells for: DANA-LOZIER (LOZIER #10)
Searching Active Wells for: SAGRERA HEIRS
Searching Active Wells for: TP SU68;WRIGHT
Searching Active Wells for: WX E RB SU6;KAUFFMAN
Searching Active Wells for: WX G RB SU6;KAUFFMAN
Searching Active Wells for: TP SU68;KIRKPATRICK
Searching Active Wells for: CRIS 8 RB SUA;SAGRERA HEIRS
Searching Active Wells for: 13400 RC SUA;SAGRERA HEIRS-KTM
Searching Active Wells for: MARDIS I 1-35
Searching Active Wells for: MARDIS "I" 1-35
Searching Active Wells for: GRIFFIN "E"
Searching Active Wells for: JENKINS "C"
Searching Active Wells for: THREE "J" RANCH
Searching Active Wells for: JENKINS
Searching Active Wells for: HUNTER - KONE


18

# Concat and Modify Results

## Concat all GeoSpatial Wells to 1 DataFrame

In [33]:
hbpwells = pd.concat(listofresults)
hbpwells['Active Well Name & Number'] = hbpwells["LeaseName"] + " " + hbpwells['WellNo'].apply(lambda x: str(x))

In [34]:
hbpwells.shape

(75, 128)

## Getting Minimum Production Date for Each Active Well API

In [35]:
minstartprod = hbpwells[['APIUWI', 'FstPrdDate']].groupby(['APIUWI']).min()
minstartprod.rename(columns = {"FstPrdDate": 'MinFrstPrdDate'}, inplace = True)


minstartprod.head()

,MinFrstPrdDate
APIUWI,
15151010030000,1986-04-01
15151217600000,1988-02-01
15151218040000,1987-11-01
15151218670000,1988-02-01
15151218680000,1988-02-01
15151218970000,1988-09-01
17017324210000,1996-10-01
17017324220000,1996-08-01
17017358330000,2014-12-01


## Left Joining Results with Minimum Production Dates

In [36]:
hbpwells = pd.merge(left = hbpwells, right = minstartprod, on = 'APIUWI', how = "left")

hbpwells.head()

,APIUWI,OpAlias,LeaseName,WellNo,County,Reservoir,ProdType,ProdStatus,DrillType,TD,...,FulLqBFac,FulGsEURMo,FulGasBFac,OtherNo,geometry,Reference Inactive API,Reference Inactive Well Name,Reference Inactive Well Last Prod Date,Active Well Name & Number,MinFrstPrdDate
0,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.99,88.0,1.51,11905524400000,POINT (1235260.589 4023024.078),35119227310000,LOZIER 13,2001-12-01,LOZIER 2,1979-01-01
1,35119225880000,STACY OPERATING LLC,STATE PENNEY,4,PAYNE (OK),"RED FORK,PRUE,SKINNER,MISS",OIL,ACTIVE,V,4164.0,...,0.26,161.0,1.69,11902114900371,POINT (1235871.294 4023615.964),35119227310000,LOZIER 13,2001-12-01,STATE PENNEY 4,1985-02-01
2,35119208510000,STACY OPERATING LLC,LOZIER,1,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3940.0,...,2.00,88.0,1.51,11905524400000,POINT (1235184.467 4023424.147),35119227310000,LOZIER 13,2001-12-01,LOZIER 1,1979-01-01
3,35119225810000,STACY OPERATING LLC,BUTCHER-JAMES,4,PAYNE (OK),PRUE,OIL,ACTIVE,V,3910.0,...,1.94,166.0,1.69,11902780700000,POINT (1235453.446 4022844.164),35119227310000,LOZIER 13,2001-12-01,BUTCHER-JAMES 4,1985-06-01
4,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.99,88.0,1.51,11905524400000,POINT (1235260.589 4023024.078),35119226530000,LOZIER (TWIN) 12,2001-12-01,LOZIER 2,1979-01-01
5,35119225880000,STACY OPERATING LLC,STATE PENNEY,4,PAYNE (OK),"RED FORK,PRUE,SKINNER,MISS",OIL,ACTIVE,V,4164.0,...,0.26,161.0,1.69,11902114900371,POINT (1235871.294 4023615.964),35119226530000,LOZIER (TWIN) 12,2001-12-01,STATE PENNEY 4,1985-02-01
6,35119208510000,STACY OPERATING LLC,LOZIER,1,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3940.0,...,2.00,88.0,1.51,11905524400000,POINT (1235184.467 4023424.147),35119226530000,LOZIER (TWIN) 12,2001-12-01,LOZIER 1,1979-01-01
7,35119010430000,STACY OPERATING LLC,BUTCHER-JAMES,2,PAYNE (OK),None,OIL,ACTIVE,V,3950.0,...,1.27,381.0,1.52,11902780700000,POINT (1235672.356 4022660.054),35119226530000,LOZIER (TWIN) 12,2001-12-01,BUTCHER-JAMES 2,1970-01-01
8,35119225810000,STACY OPERATING LLC,BUTCHER-JAMES,4,PAYNE (OK),PRUE,OIL,ACTIVE,V,3910.0,...,1.94,166.0,1.69,11902780700000,POINT (1235453.446 4022844.164),35119226530000,LOZIER (TWIN) 12,2001-12-01,BUTCHER-JAMES 4,1985-06-01
9,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.99,88.0,1.51,11905524400000,POINT (1235260.589 4023024.078),35119225680000,LOZIER (TWIN) 11,2001-12-01,LOZIER 2,1979-01-01


###  Create Function to Get Time Difference and See if Wells that Are Active Could've Held Well/Lease based on relativety to last production date

In [90]:
def withinOneYear(min_first_prod, last_prod_inactivewell, refwellaname):
    '''
    min_first_prod: earliest first prod date of active well
    last_prod_inactivewell: last prod date of inactive well
    refwellname: name of inactive well
    return: string to tell whether or not well started prod within 1 year of inactive well going offline
    
    '''
    
    #try to convert value to datetime - prevents error if there is not string date in row
    try:
        min_first_prod = dt.datetime.strptime(min_first_prod, "%Y-%m-%d")
        last_prod_inactivewell = dt.datetime.strptime(last_prod_inactivewell, "%Y-%m-%d")
    except:
        return "Not Applicable - No active Wells in 0.5 mi radius"
        
        
    #subtracting 2 dates to get difference of days
    deltaT = min_first_prod - last_prod_inactivewell
    
    if deltaT.days <=365:
        return f'Well began producing before or within 1 year of {refwellaname} going offline on {last_prod_inactivewell.date()}'
    else:
        return f"Well did NOT begin producing within 1 year of {refwellaname} last producing, which was {last_prod_inactivewell.date()}"
    
    
    
    
def IswithinOneYear(min_first_prod, last_prod_inactivewell, refwellaname):
    '''
    min_first_prod: earliest first prod date of active well
    last_prod_inactivewell: last prod date of inactive well
    refwellname: name of inactive well
    return: Boolean value of whether well started producing within 1 year or not
    '''

    #try to convert value to datetime - prevents error if there is not string date in row
    try:
        min_first_prod = dt.datetime.strptime(min_first_prod, "%Y-%m-%d")
        last_prod_inactivewell = dt.datetime.strptime(last_prod_inactivewell, "%Y-%m-%d")
    except:
        return "Not Applicable - No active Wells in 0.5 mi radius"


    #subtracting 2 dates to get difference of days
    deltaT = min_first_prod - last_prod_inactivewell

    if deltaT.days <=365:
        return True
    else:
        return False



#### Previewing Function Results

In [91]:
hbpwells["Well Active within 1 Year of Last Prod Date"] = hbpwells.apply(lambda x: withinOneYear(x['MinFrstPrdDate'], x['Reference Inactive Well Last Prod Date'], x["Reference Inactive Well Name"]), axis = 1)
hbpwells["Well Potentially HBP Lease?"] = hbpwells.apply(lambda x: IswithinOneYear(x['MinFrstPrdDate'], x['Reference Inactive Well Last Prod Date'], x["Reference Inactive Well Name"]), axis = 1)

hbpwells.head()

,APIUWI,OpAlias,LeaseName,WellNo,County,Reservoir,ProdType,ProdStatus,DrillType,TD,...,FulGasBFac,OtherNo,geometry,Reference Inactive API,Reference Inactive Well Name,Reference Inactive Well Last Prod Date,Active Well Name & Number,MinFrstPrdDate,Well Active within 1 Year of Last Prod Date,Well Potentially HBP Lease?
0,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.51,11905524400000,POINT (1235260.589 4023024.078),35119227310000,LOZIER 13,2001-12-01,LOZIER 2,1979-01-01,Well began producing before or within 1 year o...,True
1,35119225880000,STACY OPERATING LLC,STATE PENNEY,4,PAYNE (OK),"RED FORK,PRUE,SKINNER,MISS",OIL,ACTIVE,V,4164.0,...,1.69,11902114900371,POINT (1235871.294 4023615.964),35119227310000,LOZIER 13,2001-12-01,STATE PENNEY 4,1985-02-01,Well began producing before or within 1 year o...,True
2,35119208510000,STACY OPERATING LLC,LOZIER,1,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3940.0,...,1.51,11905524400000,POINT (1235184.467 4023424.147),35119227310000,LOZIER 13,2001-12-01,LOZIER 1,1979-01-01,Well began producing before or within 1 year o...,True
3,35119225810000,STACY OPERATING LLC,BUTCHER-JAMES,4,PAYNE (OK),PRUE,OIL,ACTIVE,V,3910.0,...,1.69,11902780700000,POINT (1235453.446 4022844.164),35119227310000,LOZIER 13,2001-12-01,BUTCHER-JAMES 4,1985-06-01,Well began producing before or within 1 year o...,True
4,35119209100000,STACY OPERATING LLC,LOZIER,2,PAYNE (OK),"MISENER SAND,MISS LIME",OIL,ACTIVE,V,3804.0,...,1.51,11905524400000,POINT (1235260.589 4023024.078),35119226530000,LOZIER (TWIN) 12,2001-12-01,LOZIER 2,1979-01-01,Well began producing before or within 1 year o...,True


# Exporting Table to File 

In [92]:
hbpwells.to_excel("Output/Source Table - Potential HBP Wells.xlsx", index = False)

In [78]:
print(hbpwells.columns)

Index(['APIUWI', 'OpAlias', 'LeaseName', 'WellNo', 'County', 'Reservoir',
       'ProdType', 'ProdStatus', 'DrillType', 'TD',
       ...
       'FulGasBFac', 'OtherNo', 'geometry', 'Reference Inactive API',
       'Reference Inactive Well Name',
       'Reference Inactive Well Last Prod Date', 'Active Well Name & Number',
       'MinFrstPrdDate', 'Well Active within 1 Year of Last Prod Date',
       'Well Potentially HBP Lease?'],
      dtype='object', length=131)


In [87]:
abridgedCols = ['Reference Inactive API','Reference Inactive Well Name', 'Reference Inactive Well Last Prod Date', 'County', 'APIUWI', 'OpAlias', 'LeaseName', 'WellNo', "FstPrdDate", 'Well Active within 1 Year of Last Prod Date', 'Well Potentially HBP Lease?']

In [88]:
abridged = hbpwells[abridgedCols]

In [89]:
abridged.to_excel('Output/Abridged Table.xlsx', index = False, sheet_name = 'AbridgedTable')